<a href="https://colab.research.google.com/github/wesley28/ProcessamentoDigitalDeImagens/blob/main/Redimensionamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import math
from collections import Counter


class ImagemBPM:

  def __init__(self, tipo='', largura=0, altura=0, valor_maximo = 1, colorido=False):
    self.tipo = tipo
    self.largura = largura
    self.altura = altura
    self.valor_maximo = valor_maximo
    self.colorido = colorido


  def _read(self, path):
    return open(path, "r").read().split('\n')


  def extrair_imagem(self, path):
    ls_imagem = self._read(path)

    if self.colorido:
      self.tipo = ls_imagem[0]
      self.largura = int(ls_imagem[2].split()[0]) * 3
      self.altura = int(ls_imagem[2].split()[1])
      self.valor_maximo = int(ls_imagem[3])

      conteudo = ls_imagem[5:]
      conteudo = [i.split(' ') for i in conteudo]
      np_conteudo = np.array(conteudo)
      np_conteudo = np.concatenate(np_conteudo)
      self.conteudo = np.delete(np_conteudo, np.where(np_conteudo == ''))

    else:
      self.tipo = ls_imagem[0]
      self.largura = int(ls_imagem[1].split()[0])
      self.altura = int(ls_imagem[1].split()[1])
      self.valor_maximo = int(ls_imagem[2])
      self.conteudo = np.array([int(i) if i != '' else 0 for i in ls_imagem[2 : self.altura * self.largura + 2]])


  def compactar(self, fator):
    assert self.altura % fator == 0, 'O fator de compactação precisa ser múltiplo da altura'
    assert self.largura % fator == 0, 'O fator de compactação precisa ser múltiplo da largura'

    self.fator = fator
    self.nova_altura = int(self.altura / self.fator)
    self.nova_largura = int(self.largura / self.fator)

    rs_conteudo = self.conteudo.reshape(int((self.altura * self.largura) / (self.fator * self.fator)), self.fator * self.fator)

    conteudo_compactado = []
    for agrupamento in self._gen_agrupamento_pixels(rs_conteudo):
      conteudo_compactado.append(agrupamento)

    self.conteudo_compactado = conteudo_compactado


  def _gen_agrupamento_pixels(self, ls_conteudo):
    for valores in ls_conteudo:
      #yield max(valores)
      yield min(valores)
      #yield round(sum(valores) / len(valores))
      #c = Counter(valores)
      #yield c.most_common(1)[0][0]


  def redimensionar(self, nv_largura, nv_altura):
    self.nv_largura = nv_largura
    self.nv_altura = nv_altura
    matriz_conteudo = self.conteudo.reshape(self.largura, self.altura)
    matriz_conteudo_copia = matriz_conteudo.copy()

    diff_largura = nv_largura - self.largura
    if diff_largura != 0:
      lt_linhas = list(range(self.largura))
      random.shuffle(lt_linhas)

      if diff_largura > 0:
        linhas_add = lt_linhas[:diff_largura]
        for id in linhas_add:
          matriz_conteudo_copia = np.insert(matriz_conteudo_copia, self.largura * id, matriz_conteudo[id, :])

      else:
        linhas_rmv = lt_linhas[diff_largura:]
        index_rmv = []
        for row in linhas_rmv:
          index_rmv.append([i for i in range(row * self.largura, (row + 1) * self.largura)])
        index_rmv = list(np.concatenate(index_rmv).flat)
        matriz_conteudo_copia = np.delete(matriz_conteudo_copia, index_rmv)

      matriz_conteudo_copia = matriz_conteudo_copia.reshape(nv_largura, self.altura)
      self.conteudo_redimensionado = matriz_conteudo_copia.flatten()

    matriz_conteudo_copia2 = matriz_conteudo_copia.copy()
    diff_altura = nv_altura - self.altura
    if diff_altura != 0:
      lt_colunas = list(range(self.altura))
      random.shuffle(lt_colunas)
      if diff_altura > 0:
        colunas_add = lt_colunas[:diff_altura]

        for id in colunas_add:
          matriz_conteudo_copia2 = np.insert(matriz_conteudo_copia2, self.altura * id, matriz_conteudo_copia[:, id])

        self.conteudo_redimensionado = matriz_conteudo_copia2

      else:
        colunas_rmv = lt_colunas[diff_altura:]
        index_rmv = []
        for col in colunas_rmv:
          index_rmv.append([i for i in range(col * self.nv_largura, (col + 1) * self.nv_largura)])
        index_rmv = list(np.concatenate(index_rmv).flat)
        matriz_conteudo_copia2 = np.delete(matriz_conteudo_copia2, index_rmv)

      self.conteudo_redimensionado = matriz_conteudo_copia2.flatten()



  def _gerar_cabecalho(self):
    return '{}\n{} {}\n {}\n'.format(self.tipo, self.largura, self.altura, self.valor_maximo)


  def salvar_imagem(self, nome, formato):
    with open('{}.{}'.format(nome, formato), 'w') as fd:
      fd.write(self.imagem)


  def gerar_imagem(self, colorido = False, random = True, compactado = False, redimensionar = False):

    if random:
      cabecalho = self._gerar_cabecalho()
      matriz = '\n'.join([' '.join([str(random.randint(0, self.valor_maximo)) for _ in range(self.altura * 3 if colorido else self.altura)]) for _ in range(self.largura)])
      self.imagem = cabecalho + matriz

    elif compactado:
      self.imagem = '{} \n{} {} \n{} \n{}'.format(
          self.tipo, self.nova_largura, self.nova_altura, self.valor_maximo,
          '\n'.join([str(i) + ' ' for i in self.conteudo_compactado]))

    elif redimensionar:
      self.imagem = '{}\n{} {}\n{}'.format(
          self.tipo, self.nv_largura, self.nv_altura,
          '\n'.join([str(i) for i in self.conteudo_redimensionado.tolist()]))

    else:
      self.imagem = '{}\n{} {}\n{}'.format(
          self.tipo, self.largura, self.altura,
          '\n'.join([str(i) for i in self.conteudo.tolist()]))


In [2]:
from collections import Counter
c = Counter([1,1,2,2,2,3])
c.most_common(1)[0][0]
#multimode([1, 1, 1,1, 2, 3, 3, 3, 3, 4])

2

In [18]:

t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.redimensionar(10, 10)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
# Salva a imagem redimensionada como img_cinza_10xmenor.pgm
t1.salvar_imagem('img_cinza_10xmenor.', 'pgm')

In [5]:
t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.redimensionar(480, 320)
#t1.compactar(4)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
t1.salvar_imagem('img_cinza_480_320', 'pgm')
#print(t1.nova_altura, t1.nova_largura, t1.tipo, len(t1.conteudo_compactado))
#len(t1.conteudo_compactado)

#assert t1.altura % 11 == 0, 'O fator de compactação precisa ser múltiplo da altura'

In [6]:
t1 = ImagemBPM()
t1.extrair_imagem("/content/Entrada_EscalaCinza.txt")
t1.redimensionar(1280, 720)
t1.gerar_imagem(random=False, compactado=False, redimensionar=True)
t1.salvar_imagem('img_cinza_1280_720', 'pgm')

In [29]:
import numpy as np
import cv2

class ImagemBPM:
    def __init__(self, arquivo):
        self.arquivo = arquivo

        with open(arquivo, 'r') as f:
            ls_imagem = f.readlines()

        self.tipo = ls_imagem[0]
        self.largura = int(ls_imagem[1].split()[0])
        self.altura = int(ls_imagem[1].split()[1])
        self.conteudo = np.array([int(i) if i != '' else 0 for i in ls_imagem[2 : self.altura * self.largura + 1]])

    def _read(self, path):
        with open(path, 'r') as f:
            conteudo = f.readlines()
        return conteudo

    def salvar_imagem(self, nome, formato):
        if formato == 'pgm':
            imagem = '{}\n{} {}\n{}'.format(
                self.tipo, self.largura, self.altura,
                '\n'.join([str(i) for i in self.conteudo.tolist()]))
            with open(nome, 'w') as f:
                f.write(imagem)
        elif formato == 'png':
            cv2.imwrite(nome + '.png', self.conteudo)
        else:
            raise ValueError('Formato de imagem inválido')

def txt_para_pgm(arquivo_txt, arquivo_pgm):
    imagem = ImagemBPM(arquivo_txt)
    imagem.salvar_imagem(arquivo_pgm, 'pgm')

if __name__ == '__main__':
    txt_para_pgm('/content/Entrada_EscalaCinza.txt', 'img_cinza_1920_1080.pgm')

In [32]:
#4K
import cv2
img = cv2.imread("Entrada_EscalaCinza.txt", cv2.IMREAD_GRAYSCALE)
new_img = cv2.resize(img, (3840, 2160), interpolation=cv2.INTER_LINEAR)
cv2.imwrite("imag_cinza_4k.pgm", new_img)


True

In [34]:
#8K
img = cv2.imread("Entrada_EscalaCinza.txt", cv2.IMREAD_GRAYSCALE)
new_img = cv2.resize(img, (7680, 4320), interpolation=cv2.INTER_LINEAR)
cv2.imwrite("imag_cinza_8k.pgm", new_img)

True

In [27]:
op = open("/content/Entrada_EscalaCinza.txt", "r").read().split('\n')
op

['P2',
 '800 800',
 '255',
 '110',
 '110',
 '110',
 '110',
 '110',
 '110',
 '110',
 '110',
 '110',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '109',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '107',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '106',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105',
 '105

In [ ]:
def other_math(number):
  print('do the math with:', number)

def gen_squares_in_interval(x, y):
  for i in range(x, y):
    yield i*i

if __name__ == '__main__':

  x, y = (10, 20)

  #n = []
  number = gen_squares_in_interval(x, y)
  #for number in gen_squares_in_interval(x, y):
  #  n.append(number)

  #gen = gen_squares_in_interval(x, y)


In [ ]:
next(number)

100

In [ ]:
# local insert row: len(row) * (id row )
array = [
    [11, 12, 13],
    [4, 5, 6],
    [7, 8, 9]
]

a = np.array(array)
#b = a[[1,2], :] np.delete(a, index)
np.insert(a, 3, [10,11,12]).reshape(3, 4).flatten()
a
np.delete(a, 2)
np.delete(a.reshape(3,3), [0,2, 4])

array([12,  4,  6,  7,  8,  9])

In [ ]:
l = [0, 2, 5, 7, 11]
l2 = []
for row in l:
  l2.append([i for i in range(row * 10, (row + 1) * 10)])
#[range(row * 10, (row + 1) * 10) for row in l]
l2
list(np.concatenate(l2).flat)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119]